In [1]:
# ------------------------------------------------------------
# 📧 ASC Email Composer — Colab Edition
# Author: Nana Kwaku Amoako
# Date: July 2025
# Description:
# This interactive tool allows you to compose, preview, and send
# branded emails with embedded images and file attachments using
# markdown-style formatting.
# ------------------------------------------------------------

# FOLLOW ALL INSTRUCTIONS.
## DO NOT RUN ALL THE CODE CHUNKS AT ONCE. DO IT STEP BY STEP

In [ ]:
!pip install ipywidgets

# 📤 Step 1: File Upload + Column Mapping + Email Validation

In [ ]:
import pandas as pd
import re
from ipywidgets import FileUpload, Dropdown, VBox, Output, Label, Button, HTML, HBox
from IPython.display import display, clear_output

# Widgets
upload_widget = FileUpload(accept='.csv', multiple=False)
output_preview = Output()
column_selector_name = Dropdown(description="Name column")
column_selector_email = Dropdown(description="Email column")
submit_columns_btn = Button(description="✅ Confirm Columns", button_style='success')
validation_output = Output()

# Global DataFrame
df = pd.DataFrame()

def handle_upload(change):
  clear_output(wait=True)
  output_preview.clear_output()
  global df
  if upload_widget.value:
      try:
          file_info = list(upload_widget.value.values())[0]
          content = file_info['content']
          df = pd.read_csv(pd.io.common.BytesIO(content))

          with output_preview:
              print("✅ CSV File Uploaded Successfully!")
              print(f"📊 Found {len(df)} rows and {len(df.columns)} columns")

              # Auto-detect likely name and email columns
              name_candidates = [col for col in df.columns if any(word in col.lower() for word in ['name', 'full', 'first', 'last'])]
              email_candidates = [col for col in df.columns if 'email' in col.lower() or 'mail' in col.lower()]

              if name_candidates:
                  print(f"💡 Auto-detected Name column: '{name_candidates[0]}'")
              if email_candidates:
                  print(f"💡 Auto-detected Email column: '{email_candidates[0]}'")
              if not name_candidates or not email_candidates:
                  print("⚠️ Couldn't auto-detect name/email columns. Please select manually below.")

              print("\n👀 Preview of your data:")
              display(df.head())

              print("\n📝 Available placeholders for your email template:")
              for col in df.columns:
                  print(f"   {{{{ {col} }}}}")
              print("\n💡 Copy these exact placeholder names to use in Step 2!")

          # Update dropdown options
          columns = df.columns.tolist()
          column_selector_name.options = columns
          column_selector_email.options = columns

          # Auto-select if we found candidates
          if name_candidates:
              column_selector_name.value = name_candidates[0]
          if email_candidates:
              column_selector_email.value = email_candidates[0]

          display(VBox([
              output_preview,
              HTML(value="<hr style='border: 2px solid #007acc; margin: 20px 0;'>"),
              HTML(value="<h3 style='color: #007acc;'>📋 Step 1.2: Select Your Columns</h3>"),
              HTML(value="<p style='color: #666; margin: 10px 0;'>Choose which columns contain the name and email addresses for your recipients:</p>"),
              HBox([column_selector_name, column_selector_email]),
              submit_columns_btn,
              validation_output
          ]))

      except Exception as e:
          with output_preview:
              print("❌ Error reading CSV file!")
              print(f"Error details: {str(e)}")
              print("\n💡 Troubleshooting steps:")
              print("   ✓ Make sure your file is a valid CSV format")
              print("   ✓ Ensure the file has column headers in the first row")
              print("   ✓ Check that the file isn't corrupted or empty")
              print("   ✓ Try saving your Excel file as CSV if needed")

  # Detect available placeholders from columns
  if not df.empty:
      placeholder_candidates = [col for col in df.columns if col not in [column_selector_name.value, column_selector_email.value]]
      st_placeholders = placeholder_candidates  # Save for Step 2 usage


def validate_email_list(name_col, email_col):
  validation_output.clear_output()
  if df.empty:
      return
  errors = []
  for idx, row in df.iterrows():
      email = str(row[email_col]).strip()
      name = str(row[name_col]).strip()
      if not re.match(r"[^@]+@[^@]+\.[^@]+", email):
          errors.append((idx+1, name, email))
  with validation_output:
      if errors:
          print("❌ Invalid email addresses found:")
          for row_num, name, email in errors:
              print(f"Row {row_num}: {name} - {email}")
      else:
          print("✅ All emails are valid!")

submit_columns_btn.on_click(lambda b: validate_email_list(
    column_selector_name.value, column_selector_email.value
))
upload_widget.observe(handle_upload, names='value')

# Display upload UI with enhanced guidance
display(HTML(value="""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; margin: 10px 0;">
<h2 style="margin: 0; text-align: center;">📧 Personal - ASC SMTP Email Service - Step 1</h2>
<p style="text-align: center; margin: 5px 0;">Upload your contact list to get started</p>
</div>
"""))

display(HTML(value="""
<div style="background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #007acc;">
<h3 style="color: #007acc; margin-top: 0;">📋 CSV File Requirements</h3>
<ul style="margin: 10px 0;">
<li><strong>Format:</strong> Must be a .csv file (not .xlsx or .xls)</li>
<li><strong>Headers:</strong> First row should contain column names</li>
<li><strong>Required columns:</strong> At least one for names and one for email addresses</li>
<li><strong>Example format:</strong></li>
</ul>
<pre style="background: #e9ecef; padding: 10px; border-radius: 4px; font-family: monospace;">
Name,Email,Event,Location
John Doe,john@example.com,Conference,New York
Jane Smith,jane@example.com,Conference,New York
</pre>
</div>
"""))

display(HTML(value="<h3 style='color: #007acc;'>📤 Step 1.1: Upload Your CSV File</h3>"))
display(upload_widget)


HTML(value='\n<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding…

HTML(value='\n<div style="background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0; border-left:…

HTML(value="<h3 style='color: #007acc;'>📤 Step 1.1: Upload Your CSV File</h3>")

FileUpload(value={}, accept='.csv', description='Upload')

# 📥 📨 Step 2 – Styled Email Composer with Preview + Test Send

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.image import MIMEImage
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
import io

# -- Globals from Step 1 --
# Assume df is already loaded, and st_placeholders contains dynamic columns
# E.g., st_placeholders = ['event', 'location', 'date']

# --- Widgets ---
sender_email_input = widgets.Text(
    placeholder='yourname@example.com', description='Email:', layout=widgets.Layout(width='100%')
)
password_input = widgets.Password(description='Password:', layout=widgets.Layout(width='100%'))

subject_input = widgets.Text(
    value="🎉 Special Invite Just for You", description='Subject:', layout=widgets.Layout(width='100%')
)

salutation_input = widgets.Textarea(
    value="Dear {{name}},", description="Salutation:", layout=widgets.Layout(width='100%', height='60px')
)

body_input = widgets.Textarea(
    value="This is a sample email body. Edit here...",
    description="Body:", layout=widgets.Layout(width='100%', height='200px')
)

signature_input = widgets.Textarea(
    value="— Your ASC Family", description="Signature:", layout=widgets.Layout(width='100%', height='50px')
)

logo_uploader = widgets.FileUpload(accept='image/*', multiple=False, description="🏢 Upload Custom Logo (Optional)")
logo_preview_output = widgets.Output()
embedded_images_uploader = widgets.FileUpload(accept='image/*', multiple=True, description="📷 Upload Embedded Images")
attachments_uploader = widgets.FileUpload(accept='.pdf,image/*', multiple=True, description="📎 Upload Attachments (≤5MB)")
preview_button = widgets.Button(description="👀 Preview Email", button_style='info')
test_button = widgets.Button(description="📨 Send Test Email", button_style='warning')
test_email_input = widgets.Text(placeholder='Enter your email', description='Test To:')
preview_output = widgets.Output()
test_output = widgets.Output()

# --- Logo Preview Handler ---
def update_logo_preview(change):
    logo_preview_output.clear_output()
    with logo_preview_output:
        if logo_uploader.value:
            import base64
            logo_file = list(logo_uploader.value.values())[0]
            logo_content = logo_file['content']
            logo_base64 = base64.b64encode(logo_content).decode('utf-8')
            logo_name = logo_file['metadata']['name']

            if logo_name.lower().endswith('.png'):
                data_url = f"data:image/png;base64,{logo_base64}"
            elif logo_name.lower().endswith(('.jpg', '.jpeg')):
                data_url = f"data:image/jpeg;base64,{logo_base64}"
            else:
                data_url = f"data:image/png;base64,{logo_base64}"

            print(f"📄 Custom logo: {logo_name}")
            display(widgets.HTML(value=f'<img src="{data_url}" style="max-width: 120px; height: auto; border: 1px solid #ddd; padding: 5px;" />'))
        else:
            print("📄 Default ASC logo will be used")
            display(widgets.HTML(value='<img src="https://raw.githubusercontent.com/nanadotam/1nri-photo/main/temp/ASC%20LOGO%20PNG.png" style="max-width: 120px; height: auto; border: 1px solid #ddd; padding: 5px;" />'))

logo_uploader.observe(update_logo_preview, names='value')

# Initialize logo preview with default
update_logo_preview(None)

# --- Markdown Formatter ---
def markdown_to_html(text):
    # Bold + Italic (***text***)
    text = re.sub(r'\*\*\*(.*?)\*\*\*', r'<strong><em>\1</em></strong>', text)
    # Bold (**text**)
    text = re.sub(r'\*\*(.*?)\*\*', r'<strong>\1</strong>', text)
    # Italic (_text_)
    text = re.sub(r'_(.*?)_', r'<em>\1</em>', text)
    # Strikethrough (~~text~~)
    text = re.sub(r'~~(.*?)~~', r'<del>\1</del>', text)
    # Monospace (`text`)
    text = re.sub(r'`(.*?)`', r'<code>\1</code>', text)
    # Horizontal Rule (---)
    text = re.sub(r'^---$', r'<hr>', text, flags=re.MULTILINE)
    # Blockquote (> text)
    text = re.sub(r'^> (.*)$', r'<blockquote>\1</blockquote>', text, flags=re.MULTILINE)
    # Links ([text](url))
    text = re.sub(r'\[([^\]]+)\]\(([^)]+)\)', r'<a href="\2">\1</a>', text)
    # Images (![alt](url))
    text = re.sub(r'!\[([^\]]*)\]\(([^)]+)\)', r'<img src="\2" alt="\1" style="max-width: 100%;">', text)
    # Unordered list (- item)
    text = re.sub(r'^- (.*)$', r'<ul><li>\1</li></ul>', text, flags=re.MULTILINE)
    # Ordered list (1. item)
    text = re.sub(r'^\d+\. (.*)$', r'<ol><li>\1</li></ol>', text, flags=re.MULTILINE)
    # Line breaks
    text = re.sub(r'\n', r'<br>', text)
    return text

# --- Dynamic HTML Email Generator ---
def generate_email_html(row, embedded_images=None):
    import base64

    # Check if custom logo is uploaded, otherwise use default
    if logo_uploader.value:
        # Use uploaded custom logo
        logo_file = list(logo_uploader.value.values())[0]
        logo_content = logo_file['content']
        logo_base64 = base64.b64encode(logo_content).decode('utf-8')
        # Detect image type
        if logo_file['metadata']['name'].lower().endswith('.png'):
            logo_url = f"data:image/png;base64,{logo_base64}"
        elif logo_file['metadata']['name'].lower().endswith(('.jpg', '.jpeg')):
            logo_url = f"data:image/jpeg;base64,{logo_base64}"
        else:
            logo_url = f"data:image/png;base64,{logo_base64}"  # Default to PNG
        logo_alt = "Custom Logo"
    else:
        # Use default ASC logo
        logo_url = "https://raw.githubusercontent.com/nanadotam/1nri-photo/main/temp/ASC%20LOGO%20PNG.png"
        logo_alt = "ASC Logo"

    def apply_placeholders(text):
        for col in df.columns:
            placeholder = f"{{{{{col}}}}}"
            value = str(row.get(col, ''))
            text = text.replace(placeholder, value)
        return markdown_to_html(text)

    salutation = apply_placeholders(salutation_input.value)
    body = apply_placeholders(body_input.value)

    embedded_html = ""
    if embedded_images:
        for idx in range(len(embedded_images)):
            embedded_html += f'<img src="cid:image{idx}" style="max-width: 100%; margin-top: 20px;" /><br/>'

    return f"""
    <html>
      <head>
        <link href="https://fonts.googleapis.com/css2?family=Playfair+Display:ital,wght@1,400&display=swap" rel="stylesheet">
      </head>
      <body style="margin: 0; padding: 0; background-color: #f8f8f8; font-family: 'Times New Roman', serif;">
        <div style="max-width: 700px; margin: 40px auto; background-color: white; padding: 14px;">
          <div style="border: 2px solid #83142A; padding: 30px;">

            <div style="text-align: center; margin-bottom: 30px;">
              <img src="{logo_url}" alt="{logo_alt}" style="max-width: 120px; height: auto;" />
            </div>

            <div style="font-family: 'Playfair Display', 'Times New Roman', serif; font-style: italic; font-size: 20px; margin-bottom: 20px; color: #262626;">
              {salutation}
            </div>

            <div style="font-size: 16px; line-height: 1.7; color: #222;">
              {body}
            </div>

            {embedded_html}

            <div style="margin-top: 30px; font-family: 'Playfair Display', 'Times New Roman', serif; font-style: italic; font-size: 16px; color: #262626;">
              {apply_placeholders(signature_input.value)}
            </div>

          </div>
        </div>
      </body>
    </html>
    """

# --- Format Button Logic ---
def insert_format(tag):
    original = body_input.value
    insert = ""

    if tag == 'bold':
        insert = "**bold text**"
    elif tag == 'italic':
        insert = "_italic text_"
    elif tag == 'bolditalic':
        insert = "***bold italic***"
    elif tag == 'strike':
        insert = "~~strikethrough~~"
    elif tag == 'mono':
        insert = "`monospace`"
    elif tag == 'hrule':
        insert = "\n---\n"
    elif tag == 'blockquote':
        insert = "> quoted text"
    elif tag == 'ulist':
        insert = "- item"
    elif tag == 'olist':
        insert = "1. item"
    elif tag == 'link':
        insert = "[link text](https://example.com)"
    elif tag == 'image':
        insert = "![alt text](https://example.com/image.png)"
    elif tag == 'linebreak':
        insert = "\n"

    # Append the formatting to the end of current content
    body_input.value = original + insert

# --- Buttons ---
bold_btn = widgets.Button(description="B", button_style='primary')
italic_btn = widgets.Button(description="I", button_style='info')
bolditalic_btn = widgets.Button(description="B+I", button_style='primary')
strike_btn = widgets.Button(description="S", button_style='warning')
mono_btn = widgets.Button(description="Mono", button_style='')
hrule_btn = widgets.Button(description="HR", button_style='')
blockquote_btn = widgets.Button(description="❝", button_style='')
ulist_btn = widgets.Button(description="•", button_style='')
olist_btn = widgets.Button(description="1.", button_style='')
link_btn = widgets.Button(description="🔗", button_style='')
image_btn = widgets.Button(description="🖼️", button_style='')
line_btn = widgets.Button(description="↵", button_style='')

# --- Bind Clicks ---
bold_btn.on_click(lambda b: insert_format('bold'))
italic_btn.on_click(lambda b: insert_format('italic'))
bolditalic_btn.on_click(lambda b: insert_format('bolditalic'))
strike_btn.on_click(lambda b: insert_format('strike'))
mono_btn.on_click(lambda b: insert_format('mono'))
hrule_btn.on_click(lambda b: insert_format('hrule'))
blockquote_btn.on_click(lambda b: insert_format('blockquote'))
ulist_btn.on_click(lambda b: insert_format('ulist'))
olist_btn.on_click(lambda b: insert_format('olist'))
link_btn.on_click(lambda b: insert_format('link'))
image_btn.on_click(lambda b: insert_format('image'))
line_btn.on_click(lambda b: insert_format('linebreak'))

# --- Preview Handler ---
def preview_email(b):
    preview_output.clear_output()
    if df.empty:
        with preview_output:
            print("⚠️ Upload a CSV first.")
        return
    email_html = generate_email_html(df.iloc[0], embedded_images_uploader.value)
    with preview_output:
        # Show logo status
        if logo_uploader.value:
            logo_name = list(logo_uploader.value.keys())[0]
            print(f"🏢 Using custom logo: {logo_name}")
        else:
            print("🏢 Using default ASC logo")

        display(widgets.HTML(value=email_html))

        if embedded_images_uploader.value:
            print(f"✅ Embedded {len(embedded_images_uploader.value)} image(s).")
        if attachments_uploader.value:
            print(f"📎 {len(attachments_uploader.value)} file attachment(s) ready.")

# --- Test Email Sender ---
def send_test_email(b):
    test_output.clear_output()
    recipient = test_email_input.value.strip()
    if not re.match(r"[^@]+@[^@]+\.[^@]+", recipient):
        with test_output:
            print("❌ Invalid test email")
        return

    msg = MIMEMultipart("related")
    msg["Subject"] = subject_input.value
    msg["From"] = sender_email_input.value
    msg["To"] = recipient

    html_content = generate_email_html(df.iloc[0], embedded_images_uploader.value)
    alt_part = MIMEMultipart("alternative")
    alt_part.attach(MIMEText(html_content, "html"))
    msg.attach(alt_part)

    for idx, (fname, filedata) in enumerate(embedded_images_uploader.value.items()):
        img = MIMEImage(filedata['content'])
        img.add_header('Content-ID', f'<image{idx}>')
        img.add_header('Content-Disposition', 'inline', filename=fname)
        msg.attach(img)

    for fname, filedata in attachments_uploader.value.items():
        if len(filedata['content']) <= 5 * 1024 * 1024:
            part = MIMEApplication(filedata['content'])
            part.add_header('Content-Disposition', 'attachment', filename=fname)
            msg.attach(part)
        else:
            with test_output:
                print(f"⚠️ Skipped large file: {fname}")

    try:
        server = smtplib.SMTP("smtp.office365.com", 587)
        server.starttls()
        server.login(sender_email_input.value, password_input.value)
        server.sendmail(sender_email_input.value, recipient, msg.as_string())
        server.quit()
        with test_output:
            print("✅ Test email sent successfully!")
    except Exception as e:
        with test_output:
            print(f"❌ Failed to send test email: {e}")

# --- Hooks ---
preview_button.on_click(preview_email)
test_button.on_click(send_test_email)

# --- Display UI with enhanced organization ---
display(widgets.HTML(value="""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; margin: 10px 0;">
<h2 style="margin: 0; text-align: center;">✍️ Personal - ASC SMTP Email Service - Step 2</h2>
<p style="text-align: center; margin: 5px 0;">Compose and customize your email template</p>
</div>
"""))

# Quick Start Guide
display(widgets.HTML(value="""
<div style="background: #e8f5e8; padding: 15px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #28a745;">
<h3 style="color: #28a745; margin-top: 0;">🎯 Quick Start Guide</h3>
<ol style="margin: 10px 0;">
<li><strong>Email Settings:</strong> Enter your Office 365 email and password</li>
<li><strong>Subject Line:</strong> Write your email subject</li>
<li><strong>Logo (Optional):</strong> Upload your company logo or leave empty for ASC logo</li>
<li><strong>Email Content:</strong> Write your message using placeholders from Step 1</li>
<li><strong>Test First:</strong> Always send a test email to yourself before bulk sending</li>
</ol>
</div>
"""))

display(widgets.VBox([
    # Email Settings Section
    widgets.HTML(value="<hr style='border: 2px solid #dc3545; margin: 20px 0;'>"),
    widgets.HTML(value="<h3 style='color: #dc3545;'>🔐 Email Settings</h3>"),
    widgets.HTML(value="<p style='color: #666; margin: 10px 0;'>Enter your Office 365 email credentials (your password is secure and not stored):</p>"),
    sender_email_input,
    password_input,

    # Subject Section
    widgets.HTML(value="<hr style='border: 2px solid #6f42c1; margin: 20px 0;'>"),
    widgets.HTML(value="<h3 style='color: #6f42c1;'>📝 Email Subject</h3>"),
    widgets.HTML(value="<p style='color: #666; margin: 10px 0;'>Write a compelling subject line for your email:</p>"),
    subject_input,

    # Branding Section
    widgets.HTML(value="<hr style='border: 2px solid #fd7e14; margin: 20px 0;'>"),
    widgets.HTML(value="<h3 style='color: #fd7e14;'>🎨 Branding & Content</h3>"),
    widgets.HTML(value="<p style='color: #666; margin: 10px 0;'>Customize your email's appearance and content:</p>"),
    logo_uploader,
    logo_preview_output,
    widgets.HTML(value="<small style='color: #666;'>💡 Leave empty to use default ASC logo. Recommended size: 120px width</small>"),

    # Salutation Section
    widgets.HTML(value="<br><h4 style='color: #fd7e14;'>👋 Salutation</h4>"),
    widgets.HTML(value="<p style='color: #666; margin: 5px 0;'>How to greet your recipients (use placeholders from Step 1):</p>"),
    salutation_input,

    # Formatting Toolbar
    widgets.HTML(value="<br><h4 style='color: #fd7e14;'>🔧 Formatting Toolbar</h4>"),
    widgets.HTML(value="<p style='color: #666; margin: 5px 0;'>Click buttons to add formatting to your email body:</p>"),
    widgets.HBox([
        bold_btn, italic_btn, bolditalic_btn, strike_btn, mono_btn,
        hrule_btn, blockquote_btn, ulist_btn, olist_btn, link_btn, image_btn, line_btn
    ]),

    # Email Body
    widgets.HTML(value="<br><h4 style='color: #fd7e14;'>📄 Email Body</h4>"),
    widgets.HTML(value="""
    <div style="background: #fff3cd; padding: 10px; border-radius: 5px; margin: 5px 0;">
    <strong>💡 Formatting Tips:</strong>
    <ul style="margin: 5px 0;">
    <li>Use placeholders like {{Name}}, {{Email}}, {{Event}} from your CSV</li>
    <li>Use **text** for bold, _text_ for italic</li>
    <li>Use [link text](https://example.com) for links</li>
    <li>Click formatting buttons above to add styling</li>
    </ul>
    </div>
    """),
    body_input,

    # Signature Section
    widgets.HTML(value="<br><h4 style='color: #fd7e14;'>✍️ Email Signature</h4>"),
    widgets.HTML(value="<p style='color: #666; margin: 5px 0;'>Add your closing signature:</p>"),
    signature_input,

    # Attachments Section
    widgets.HTML(value="<hr style='border: 2px solid #20c997; margin: 20px 0;'>"),
    widgets.HTML(value="<h3 style='color: #20c997;'>📎 Attachments & Media</h3>"),
    widgets.HTML(value="<p style='color: #666; margin: 10px 0;'>Add images and files to your email:</p>"),
    embedded_images_uploader,
    widgets.HTML(value="<small style='color: #666;'>Images will appear in the email body</small>"),
    attachments_uploader,
    widgets.HTML(value="<small style='color: #666;'>Files will be attached to the email (max 5MB each)</small>"),

    # Testing Section
    widgets.HTML(value="<hr style='border: 2px solid #007bff; margin: 20px 0;'>"),
    widgets.HTML(value="<h3 style='color: #007bff;'>🧪 Testing & Preview</h3>"),
    widgets.HTML(value="""
    <div style="background: #d1ecf1; padding: 10px; border-radius: 5px; margin: 10px 0;">
    <strong>⚠️ Important:</strong> Always preview and test your email before sending to all recipients!
    </div>
    """),
    widgets.HBox([preview_button, test_button]),
    widgets.HTML(value="<p style='color: #666; margin: 5px 0;'>Enter your email address for testing:</p>"),
    test_email_input,
    preview_output,
    test_output
]))


HTML(value='\n<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding…

HTML(value='\n<div style="background: #e8f5e8; padding: 15px; border-radius: 8px; margin: 10px 0; border-left:…

# 📤 Step 3: Final Checks & Bulk Send

In [ ]:
from tqdm.notebook import tqdm
import time
import smtplib
import re
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from ipywidgets import Button, Output, HTML, VBox, HBox
from IPython.display import display

# --- Send Button + Output Widgets ---
send_all_button = Button(description="🚀 Send All Emails", button_style='danger')
stop_button = Button(description="⏹️ Stop Sending", button_style='warning', disabled=True)
confirm_button = Button(description="⚠️ Confirm Send to ALL Recipients", button_style='danger', disabled=True)
cancel_button = Button(description="❌ Cancel", button_style='')
send_output = Output()
confirmation_output = Output()

# --- Global flags ---
stop_sending = False
confirmed = False

# --- Confirmation Functions ---
def show_confirmation(b):
    global confirmed
    confirmed = False
    confirmation_output.clear_output()

    if df.empty:
        with confirmation_output:
            print("❌ No CSV data loaded. Please complete Step 1 first.")
        return

    recipient_count = len(df)

    with confirmation_output:
        print(f"⚠️ CONFIRMATION REQUIRED")
        print(f"You're about to send {recipient_count} emails!")
        print(f"📧 From: {sender_email_input.value}")
        print(f"📝 Subject: {subject_input.value}")
        print(f"👥 Recipients: {recipient_count}")

        if recipient_count > 50:
            print("\n🚨 WARNING: Large recipient list detected!")
            print("   Consider breaking this into smaller batches")
            print("   Sending too many emails at once may trigger spam filters")

        print("\n💡 This action cannot be undone!")
        print("   Make sure you've tested your email first")
        print("   Double-check your recipient list")

    # Enable/disable buttons
    send_all_button.disabled = True
    confirm_button.disabled = False
    cancel_button.disabled = False

def confirm_send(b):
    global confirmed
    confirmed = True
    confirmation_output.clear_output()

    with confirmation_output:
        print("✅ Confirmed! Starting bulk email send...")

    # Disable confirmation buttons, proceed with sending
    confirm_button.disabled = True
    cancel_button.disabled = True
    send_bulk_emails(None)

def cancel_send(b):
    global confirmed
    confirmed = False
    confirmation_output.clear_output()

    with confirmation_output:
        print("❌ Send cancelled. You can modify your email and try again.")

    # Re-enable send button, disable confirmation buttons
    send_all_button.disabled = False
    confirm_button.disabled = True
    cancel_button.disabled = True

# --- Stop Function ---
def stop_sending_emails(b):
    global stop_sending
    stop_sending = True
    with send_output:
        print("\n🛑 Stop requested... finishing current email and stopping.")

# --- Bulk Email Function ---
def send_bulk_emails(b):
    global stop_sending
    stop_sending = False

    send_output.clear_output()
    if df.empty:
        with send_output:
            print("❌ No CSV data loaded. Please complete Step 1 first.")
        return

    # Pre-flight checks
    if not sender_email_input.value or not password_input.value:
        with send_output:
            print("❌ Please enter your email credentials in Step 2.")
        return

    # Enable stop button, disable other buttons
    send_all_button.disabled = True
    confirm_button.disabled = True
    cancel_button.disabled = True
    stop_button.disabled = False

    name_col = column_selector_name.value
    email_col = column_selector_email.value
    total_recipients = len(df)

    failed_list = []
    sent_count = 0
    start_time = time.time()

    try:
        with send_output:
            print("🔗 Connecting to email server...")

        server = smtplib.SMTP("smtp.office365.com", 587)
        server.starttls()
        server.login(sender_email_input.value, password_input.value)

        with send_output:
            print("✅ Successfully connected to email server!")
            print("📊 Email Campaign Details:")
            print(f"   📧 From: {sender_email_input.value}")
            print(f"   📝 Subject: {subject_input.value}")
            print(f"   👥 Total Recipients: {total_recipients}")
            print("📨 Starting email send... (Click 'Stop Sending' to cancel)\n")

        for index, row in tqdm(df.iterrows(), total=len(df), desc="Sending emails"):
            # Check if stop was requested
            if stop_sending:
                with send_output:
                    print(f"\n🛑 Sending stopped by user at {sent_count}/{total_recipients} emails sent.")
                break

            name = str(row[name_col]).strip()
            recipient = str(row[email_col]).strip()

            # Validate email format
            if not re.match(r"[^@]+@[^@]+\.[^@]+", recipient):
                failed_list.append((name, recipient, "Invalid email format"))
                continue

            try:
                # Create email message
                msg = MIMEMultipart("alternative")
                msg["Subject"] = subject_input.value
                msg["From"] = sender_email_input.value
                msg["To"] = recipient
                msg.attach(MIMEText(generate_email_html(row), "html"))

                # Send email
                server.sendmail(sender_email_input.value, recipient, msg.as_string())
                sent_count += 1

                # Progress update every 10 emails
                if sent_count % 10 == 0:
                    elapsed_time = time.time() - start_time
                    avg_time_per_email = elapsed_time / sent_count
                    remaining_emails = total_recipients - sent_count
                    estimated_time_remaining = remaining_emails * avg_time_per_email

                    with send_output:
                        print(f"📈 Progress: {sent_count}/{total_recipients} sent ({(sent_count/total_recipients)*100:.1f}%)")
                        print(f"⏱️ Estimated time remaining: {estimated_time_remaining/60:.1f} minutes")

                time.sleep(0.5)  # Delay to avoid spam filters

            except smtplib.SMTPRecipientsRefused:
                failed_list.append((name, recipient, "Recipient email refused by server"))
                continue
            except smtplib.SMTPDataError as e:
                failed_list.append((name, recipient, f"SMTP Data Error: {str(e)}"))
                continue
            except Exception as e:
                failed_list.append((name, recipient, f"Unexpected error: {str(e)}"))
                continue

        server.quit()

        # Final results
        elapsed_time = time.time() - start_time
        success_rate = (sent_count / total_recipients) * 100 if total_recipients > 0 else 0

        with send_output:
            print("\n" + "="*50)
            print("📊 CAMPAIGN SUMMARY")
            print("="*50)
            print(f"✅ Successfully sent: {sent_count}/{total_recipients} emails ({success_rate:.1f}%)")
            print(f"⏱️ Total time: {elapsed_time/60:.1f} minutes")
            print(f"📧 Average: {elapsed_time/sent_count:.1f} seconds per email" if sent_count > 0 else "")

            if failed_list:
                print(f"\n❌ Failed to send: {len(failed_list)} emails")
                print("Failed recipients:")
                for entry in failed_list:
                    print(f"   • {entry[0]} ({entry[1]}): {entry[2]}")

                # Suggest retry for certain errors
                retry_candidates = [entry for entry in failed_list if "timeout" in entry[2].lower() or "connection" in entry[2].lower()]
                if retry_candidates:
                    print(f"\n💡 {len(retry_candidates)} failures may be due to temporary network issues. Consider retrying those recipients.")

            if not stop_sending and sent_count == total_recipients:
                print("\n🎉 Campaign completed successfully!")

    except smtplib.SMTPAuthenticationError:
        with send_output:
            print("❌ Authentication failed!")
            print("💡 Please check your email and password.")
            print("   Make sure you're using an Office 365 account.")
            print("   You may need to enable 'Less secure app access' or use an App Password.")
    except smtplib.SMTPConnectError:
        with send_output:
            print("❌ Cannot connect to email server!")
            print("💡 Please check your internet connection.")
    except Exception as e:
        with send_output:
            print(f"🚨 Unexpected error: {str(e)}")
            print("💡 Please try again or contact support if the issue persists.")

    finally:
        # Re-enable buttons
        send_all_button.disabled = False
        stop_button.disabled = True
        confirm_button.disabled = True
        cancel_button.disabled = True

# --- Hook Up Buttons ---
send_all_button.on_click(show_confirmation)
stop_button.on_click(stop_sending_emails)
confirm_button.on_click(confirm_send)
cancel_button.on_click(cancel_send)

# --- Display Enhanced UI ---
display(HTML(value="""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; margin: 10px 0;">
<h2 style="margin: 0; text-align: center;">🚀 Personal SMTP Email Service - Step 3</h2>
<p style="text-align: center; margin: 5px 0;">Send your emails to all recipients</p>
</div>
"""))

display(HTML(value="""
<div style="background: #fff3cd; padding: 15px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #ffc107;">
<h3 style="color: #856404; margin-top: 0;">⚠️ Safety Guidelines</h3>
<ul style="margin: 10px 0; color: #856404;">
<li><strong>Test First:</strong> Always send a test email from Step 2 before bulk sending</li>
<li><strong>Check Recipients:</strong> Verify your CSV data is correct</li>
<li><strong>Small Batches:</strong> For large lists (50+), consider breaking into smaller batches</li>
<li><strong>Monitor Progress:</strong> Watch for errors and use the stop button if needed</li>
<li><strong>Backup Plan:</strong> Keep a record of failed sends for follow-up</li>
</ul>
</div>
"""))

display(VBox([
    HTML(value="<h3 style='color: #dc3545;'>🚀 Bulk Email Sending</h3>"),
    HTML(value="<p style='color: #666; margin: 10px 0;'>Click the button below to start the confirmation process:</p>"),
    HBox([send_all_button, stop_button]),
    confirmation_output,
    HTML(value="<h4 style='color: #dc3545;'>Confirmation Required</h4>"),
    HTML(value="<p style='color: #666; margin: 5px 0;'>After clicking 'Send All Emails', you'll need to confirm:</p>"),
    HBox([confirm_button, cancel_button]),
    HTML(value="<hr style='border: 1px solid #dee2e6; margin: 20px 0;'>"),
    HTML(value="<h4 style='color: #007bff;'>📊 Sending Progress & Results</h4>"),
    send_output
]))


HTML(value='\n<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding…

HTML(value='\n<div style="background: #fff3cd; padding: 15px; border-radius: 8px; margin: 10px 0; border-left:…

In [ ]:
# =============================================
# 🛠️ Crafted with care and ❤️ by Nana Amoako | July 2025
# For: Personalized ASC Email Campaign Project
# Features:
#  → Markdown-based email editor
#  → Embedded images + attachments
#  → Real-time preview + test email support
# =============================================
